In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!sudo apt update

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,103 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,606 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRel

In [21]:
from pyspark.sql import SparkSession
# Create Spark Session in localhost
spark = SparkSession.builder.master("local").\
    appName("ITESO-DecisionTree").\
    getOrCreate()

# Set spark context
sc = spark.sparkContext
sc.setLogLevel("ERROR")

iris_df = spark.read.format("csv").\
    option("header", "true").\
    option("mode", "permissive").\
    option("path", "/content/drive/MyDrive/Colab Notebooks (1)/datasets/iris_dataset.csv").\
    load()
iris_df.show(n=10, truncate=False)

+---+-------------+------------+-------------+------------+-----------+
|Id |SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|Species    |
+---+-------------+------------+-------------+------------+-----------+
|1  |5.1          |3.5         |1.4          |0.2         |Iris-setosa|
|2  |4.9          |3.0         |1.4          |0.2         |Iris-setosa|
|3  |4.7          |3.2         |1.3          |0.2         |Iris-setosa|
|4  |4.6          |3.1         |1.5          |0.2         |Iris-setosa|
|5  |5.0          |3.6         |1.4          |0.2         |Iris-setosa|
|6  |5.4          |3.9         |1.7          |0.4         |Iris-setosa|
|7  |4.6          |3.4         |1.4          |0.3         |Iris-setosa|
|8  |5.0          |3.4         |1.5          |0.2         |Iris-setosa|
|9  |4.4          |2.9         |1.4          |0.2         |Iris-setosa|
|10 |4.9          |3.1         |1.5          |0.1         |Iris-setosa|
+---+-------------+------------+-------------+------------+-----

In [22]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col
# Create a small dataset as a list of tuples
# Format: (label, feature1, feature2)
# Create data frame
columns = ["Id","SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm","Species"]
# Assemble the features into a single vector column


iris_df = iris_df.withColumn("SepalLengthCm", col("SepalLengthCm").cast("float")) \
                 .withColumn("SepalWidthCm", col("SepalWidthCm").cast("float")) \
                 .withColumn("PetalLengthCm", col("PetalLengthCm").cast("float")) \
                 .withColumn("PetalWidthCm", col("PetalWidthCm").cast("float")) \
                 .withColumn("Id", col("Id").cast("float"))

#label_indexer = StringIndexer(inputCol= "Id", outputCol="label")
species_indexer = StringIndexer(inputCol="Species", outputCol="SpeciesIndex")

iris_df = species_indexer.fit(iris_df).transform(iris_df)


assembler = VectorAssembler(inputCols=["Id","SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"],
outputCol="features")
data_with_features = assembler.transform(iris_df).select("SpeciesIndex", "features")
# Split the data: 80% training data and 20% testing data
train, test = data_with_features.randomSplit([0.8, 0.2], seed=13)



# Initialize and train the Decision Tree model
dt = DecisionTreeClassifier(labelCol="SpeciesIndex",
featuresCol="features")
# ============================
# TRAIN
# ============================
# Train to get the model
dt_model = dt.fit(train)
# Display model summary
print("Decision Tree model summary:{0}".\
format(dt_model.toDebugString))

# Use the trained model to make predictions on the test data
predictions = dt_model.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol="SpeciesIndex",
predictionCol="prediction")
accuracy = evaluator.evaluate(predictions,
{evaluator.metricName: "accuracy"})
print(f"Accuracy: {accuracy}")
precision = evaluator.evaluate(predictions,
{evaluator.metricName: "weightedPrecision"})
print(f"Precision: {precision}")
recall = evaluator.evaluate(predictions,
{evaluator.metricName: "weightedRecall"})
print(f"Recall: {recall}")
f1 = evaluator.evaluate(predictions,
{evaluator.metricName: "f1"})
print(f"F1 Score: {f1}")



Decision Tree model summary:DecisionTreeClassificationModel: uid=DecisionTreeClassifier_c17f8333a5a2, depth=3, numNodes=7, numClasses=3, numFeatures=5
  If (feature 3 <= 2.449999988079071)
   Predict: 0.0
  Else (feature 3 > 2.449999988079071)
   If (feature 0 <= 98.5)
    Predict: 1.0
   Else (feature 0 > 98.5)
    If (feature 3 <= 4.25)
     Predict: 1.0
    Else (feature 3 > 4.25)
     Predict: 2.0

Accuracy: 1.0
Precision: 0.9999999999999999
Recall: 0.9999999999999999
F1 Score: 0.9999999999999999
